In [8]:
import dicom
import os
import pandas as pd
import matplotlib
import tensorflow as tf

data_dir='D:/stage1/'
patients=os.listdir(data_dir)
labels_df=pd.read_csv('D:/stage1_labels.csv',index_col=0)
MODEL_NAME = 'lungcancerdetect-{}-{}.model'.format(1e-7, '2conv-basic')
much_data = np.load('D:/muchdata-50-50-20.npy')
for patient in patients[:1]:
    label=labels_df.get_value(patient,'cancer')
    path=data_dir+patient
    slices=[dicom.read_file(path+'/'+s)for s in os.listdir(path)]
    slices.sort(key=lambda x:int(x.ImagePositionPatient[2]))
    print(len(slices),slices[0].pixel_array.shape)

195 (512, 512)


In [ ]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX=50
SLICE_COUNT=20

n_classes=2

x=tf.placeholder('float')
y=tf.placeholder('float')

keep_rate=0.8
keep_prob=tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,128])),
               #'W_conv4':tf.Variable(tf.random_normal([5,5,5,128,256])),
               #'W_conv5':tf.Variable(tf.random_normal([5,5,5,256,512])),
               #'W_conv6':tf.Variable(tf.random_normal([5,5,5,512,1024])),
              # 'W_conv7':tf.Variable(tf.random_normal([5,5,5,1024,2048])),
               
               'W_fc':tf.Variable(tf.random_normal([ 54080 ,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               #'b_conv3':tf.Variable(tf.random_normal([128])),
               #'b_conv4':tf.Variable(tf.random_normal([256])),
               #'b_conv5':tf.Variable(tf.random_normal([512])),
               #'b_conv6':tf.Variable(tf.random_normal([1024])),
               #'b_conv7':tf.Variable(tf.random_normal([2048])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    #conv3 = maxpool3d(conv3)
    #conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    #conv4 = maxpool3d(conv4)
    #conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    #conv5 = maxpool3d(conv5)
    #conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    #conv6 = maxpool3d(conv6)
    #conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    #conv7 = maxpool3d(conv7)
    
    

    fc = tf.reshape(conv2,[-1,  54080 ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']
    

    return output
much_data = np.load('D:/muchdata-50-50-20.npy')
train_data = much_data[-100:]
validation_data = much_data[-20:]
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        successful_runs = 0
        total_runs = 0
        hm_epochs = 10
        
        for epoch in range(hm_epochs):
            for data in train_data:
                total_runs += 1
                epoch_loss = 0
                X = data[0]
                Y = data[1]
                _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                epoch_loss += c
                successful_runs += 1
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs)    
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 10 loss: 0.0
Accuracy: 0.65
Epoch 2 completed out of 10 loss: 0.0
Accuracy: 0.65
Epoch 3 completed out of 10 loss: 0.0
Accuracy: 0.7
Epoch 4 completed out of 10 loss: 0.0
Accuracy: 0.65
Epoch 5 completed out of 10 loss: 0.0
Accuracy: 0.75
Epoch 6 completed out of 10 loss: 469666336.0
Accuracy: 0.8
Epoch 7 completed out of 10 loss: 239130592.0
Accuracy: 0.85
Epoch 8 completed out of 10 loss: 0.0
Accuracy: 0.75
Epoch 9 completed out of 10 loss: 0.0
Accuracy: 0.85
Epoch 10 completed out of 10 loss: 0.0
Accuracy: 0.9
Done. Finishing accuracy:
Accuracy: 0.85
fitment percent: 1.0


In [2]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX = 50
SLICE_COUNT = 20

n_classes = 2
batch_size = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [5]:
def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,128])),
               #'W_conv4':tf.Variable(tf.random_normal([3,3,3,128,256])),
               #                                  64 features
               'W_fc':tf.Variable(tf.random_normal([54080,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
             # 'b_conv3':tf.Variable(tf.random_normal([128])),
             # 'b_conv4':tf.Variable(tf.random_normal([256])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    #conv3 = maxpool3d(conv3)
    #conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    #conv4 = maxpool3d(conv4)

    #fc = tf.reshape(conv4,[-1, 163840])
    fc = tf.reshape(conv2,[-1, 54080])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

In [ ]:
much_data = np.load('muchdata-50-50-20.npy')
train_data = much_data[:-100]
validation_data = much_data[-297:]


def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-7).minimize(cost)
    init=tf.initialize_all_variables()
    #saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(init)
        successful_runs = 0
        total_runs = 0
        hm_epochs = 20
        
        for epoch in range(hm_epochs):
            for data in train_data:
                total_runs += 1
                try:
                    epoch_loss = 0
                    X = data[0]
                    Y = data[1]
                    #print(Y)
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_run += 1
                except Exception as e:
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs)
        #saver.save(sess,'D:/3dconvnetmodel.ckpt')


train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [5]:
#with open('D:/mysubmission_file.csv','w') as f:
    #f.write('id,label\n')

    #Restore variables from disk.
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,"D:/3dconvnetmodel.ckpt")
    print("Model restored.")            
#with open('submission_file.csv','a') as f:
    #for data in test_data:
        #img_num = data[1]
        #img_data = data[0]
        #orig = img_data
        #data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
        #model_out = model.predict([data])[0]
        #f.write('{},{}\n'.format(img_num,model_out[1]))'''

ValueError: No variables to save

In [27]:
test_data=np.load('D:/testdata-50-50-20.npy')
with open('submission_file.csv','a') as f:
    for data in test_data:
        x = data[0]
        y = data[1]
        #orig = img_data
        data = x.reshape(IMG_SIZE_PX,IMG_SIZE_PX,20)
        model_out = saver.predict([data])[0]
        f.write('{},{}\n'.format(y,model_out[1]))

AttributeError: 'Saver' object has no attribute 'predict'

In [11]:
import tflearn
import tensorflow as tf
import numpy as np
from tflearn.layers.estimator import regression

IMG_SIZE_PX=50
SLICE_COUNT=20

n_classes=2

x=tf.placeholder('float')
y=tf.placeholder('float')

keep_rate=0.8
keep_prob=tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    
    return tf.nn.maxpool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,128])),
               #'W_conv4':tf.Variable(tf.random_normal([5,5,5,128,256])),
               #'W_conv5':tf.Variable(tf.random_normal([5,5,5,256,512])),
               #'W_conv6':tf.Variable(tf.random_normal([5,5,5,512,1024])),
              # 'W_conv7':tf.Variable(tf.random_normal([5,5,5,1024,2048])),
               
               'W_fc':tf.Variable(tf.random_normal([ 54080 ,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               #'b_conv3':tf.Variable(tf.random_normal([128])),
               #'b_conv4':tf.Variable(tf.random_normal([256])),
               #'b_conv5':tf.Variable(tf.random_normal([512])),
               #'b_conv6':tf.Variable(tf.random_normal([1024])),
               #'b_conv7':tf.Variable(tf.random_normal([2048])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    #conv3 = maxpool3d(conv3)
    #conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    #conv4 = maxpool3d(conv4)
    #conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    #conv5 = maxpool3d(conv5)
    #conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    #conv6 = maxpool3d(conv6)
    #conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    #conv7 = maxpool3d(conv7)
    
    

    fc = tf.reshape(conv2,[-1,  54080 ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']
    output = regression(output, optimizer='adam', learning_rate=1e-7, loss='categorical_crossentropy', name='targets')
    
    model = tflearn.DNN(output, tensorboard_dir='log')
    #return output
    much_data = np.load('D:/muchdata-50-50-20.npy')
    train_data = much_data[-100:]
    test_data = much_data[-20:]
    if os.path.exists('{}.meta'.format(MODEL_NAME)):
        model.load(MODEL_NAME)
        print('model loaded!')
    X = np.array([i[0] for i in train_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
    Y = [i[1] for i in train_data]

    test_x = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
    test_y = [i[1] for i in test_data]

    model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

    model.save(MODEL_NAME)
convolutional_neural_network(x)

Exception: No input data! Please add an 'input_data' layer to your model (or add your input data placeholder to tf.GraphKeys.INPUTS collection).

In [5]:
import tflearn
from tflearn import conv_3d,max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

IMG_SIZE_PX=50
SLICE_COUNT=20

convnet = input_data(shape=[3, IMG_SIZE_PX, IMG_SIZE_PX,SLICE_COUNT,1])

convnet = conv_3d(convnet, 32, 3, activation='relu')
convnet = max_pool_3d(convnet, 3)

convnet = conv_3d(convnet, 64, 3, activation='relu')
convnet = max_pool_3d(convnet, 3)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = train_data[-100:]
test = train_data[-20:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)
model.save(MODEL_NAME)

AssertionError: Incoming Tensor shape must be 5-D

In [16]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX=50
SLICE_COUNT=20

n_classes=2

x=tf.placeholder('float')
y=tf.placeholder('float')

keep_rate=0.8
keep_prob=tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,128])),
               #'W_conv4':tf.Variable(tf.random_normal([5,5,5,128,256])),
               #'W_conv5':tf.Variable(tf.random_normal([5,5,5,256,512])),
               #'W_conv6':tf.Variable(tf.random_normal([5,5,5,512,1024])),
              # 'W_conv7':tf.Variable(tf.random_normal([5,5,5,1024,2048])),
               
               'W_fc':tf.Variable(tf.random_normal([ 54080 ,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               #'b_conv3':tf.Variable(tf.random_normal([128])),
               #'b_conv4':tf.Variable(tf.random_normal([256])),
               #'b_conv5':tf.Variable(tf.random_normal([512])),
               #'b_conv6':tf.Variable(tf.random_normal([1024])),
               #'b_conv7':tf.Variable(tf.random_normal([2048])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    #conv3 = maxpool3d(conv3)
    #conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    #conv4 = maxpool3d(conv4)
    #conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    #conv5 = maxpool3d(conv5)
    #conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    #conv6 = maxpool3d(conv6)
    #conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    #conv7 = maxpool3d(conv7)
    
    

    fc = tf.reshape(conv2,[-1,  54080 ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']
    

    return output
much_data = np.load('D:/muchdata-50-50-20.npy')
train_data = much_data[-100:]
validation_data = much_data[-20:]
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        successful_runs = 0
        total_runs = 0
        hm_epochs = 20
        
        for epoch in range(hm_epochs):
            for data in train_data:
                total_runs += 1
                epoch_loss = 0
                X = data[0]
                Y = data[1]
                _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                epoch_loss += c
                successful_runs += 1
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs)    
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 2 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 3 completed out of 20 loss: 0.0
Accuracy: 0.6
Epoch 4 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 5 completed out of 20 loss: 17885824.0
Accuracy: 0.7
Epoch 6 completed out of 20 loss: 0.0
Accuracy: 0.7
Epoch 7 completed out of 20 loss: 250131568.0
Accuracy: 0.8
Epoch 8 completed out of 20 loss: 0.0
Accuracy: 0.85
Epoch 9 completed out of 20 loss: 215342176.0
Accuracy: 0.95
Epoch 10 completed out of 20 loss: 0.0
Accuracy: 0.95
Epoch 11 completed out of 20 loss: 0.0
Accuracy: 0.95
Epoch 12 completed out of 20 loss: 0.0
Accuracy: 0.85
Epoch 13 completed out of 20 loss: 0.0
Accuracy: 0.95
Epoch 14 completed out of 20 loss: 0.0
Accuracy: 1.0
Epoch 15 completed out of 20 loss: 0.0
Accuracy: 0.95
Epoch 16 completed out of 20 loss: 218914496.0
Accuracy: 0.95
Epoch 17 completed out of 20 loss: 0.0
Accura

In [4]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX=50
SLICE_COUNT=20

n_classes=2

x=tf.placeholder('float')
y=tf.placeholder('float')

keep_rate=0.8
keep_prob=tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,128])),
               #'W_conv4':tf.Variable(tf.random_normal([5,5,5,128,256])),
               #'W_conv5':tf.Variable(tf.random_normal([5,5,5,256,512])),
               #'W_conv6':tf.Variable(tf.random_normal([5,5,5,512,1024])),
              # 'W_conv7':tf.Variable(tf.random_normal([5,5,5,1024,2048])),
               
               'W_fc':tf.Variable(tf.random_normal([ 54080 ,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               #'b_conv3':tf.Variable(tf.random_normal([128])),
               #'b_conv4':tf.Variable(tf.random_normal([256])),
               #'b_conv5':tf.Variable(tf.random_normal([512])),
               #'b_conv6':tf.Variable(tf.random_normal([1024])),
               #'b_conv7':tf.Variable(tf.random_normal([2048])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    #conv3 = maxpool3d(conv3)
    #conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    #conv4 = maxpool3d(conv4)
    #conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    #conv5 = maxpool3d(conv5)
    #conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    #conv6 = maxpool3d(conv6)
    #conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    #conv7 = maxpool3d(conv7)
    
    

    fc = tf.reshape(conv2,[-1,  54080 ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']
    

    return output
saver = tf.train.Saver()
much_data = np.load('D:/muchdata-50-50-20.npy')
test_data=np.load('D:/testdata-50-50-20.npy')
train_data = much_data[-990:]
validation_data = test_data[-198:]
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-7).minimize(cost)
    init=tf.initialize_all_variables()
    with tf.Session() as sess:
        sess.run(init)
        successful_runs = 0
        total_runs = 0
        hm_epochs = 20
        
        for epoch in range(hm_epochs):
            for data in train_data:
                total_runs += 1
                try:
                    epoch_loss = 0
                    X = data[0]
                    Y = data[1]
                    #print(Y)
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_run += 1
                except Exception as e:
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
            
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs)
        saver.save(sess,'D:/3dconvnetmodel.ckpt')


train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 2 completed out of 20 loss: 0.0
Accuracy: 0.7
Epoch 3 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 4 completed out of 20 loss: 0.0
Accuracy: 0.6
Epoch 5 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 6 completed out of 20 loss: 0.0
Accuracy: 0.55
Epoch 7 completed out of 20 loss: 0.0
Accuracy: 0.6
Epoch 8 completed out of 20 loss: 0.0
Accuracy: 0.55
Epoch 9 completed out of 20 loss: 0.0
Accuracy: 0.6
Epoch 10 completed out of 20 loss: 0.0
Accuracy: 0.6
Epoch 11 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 12 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 13 completed out of 20 loss: 0.0
Accuracy: 0.7
Epoch 14 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 15 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 16 completed out of 20 loss: 0.0
Accuracy: 0.65
Epoch 17 completed out of 20 loss: 0.0
Accuracy: 0.6
Epoch 18 completed out o

In [2]:
import pandas as pd
labels_df=pd.read_csv('D:/stage1_labels.csv',index_col=0)

KeyError: -100

In [38]:
import numpy as np
test_data=np.load('D:/testdata-50-50-20.npy')
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for data in enumerate(test_data[:198]):
        img_data = data[0]
        data = img_data.reshape(-1,IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT,1)
        feed_dict = {x: [data[0]]}
        classification = tf.run(tf.argmax(y, 1), feed_dict)
        print(classification)
        #data = x.reshape(-1,IMG_SIZE_PX,IMG_SIZE_PX,20,1)
        #print(sess.run(tf.argmax(y, 1), feed_dict={x: data[0]}))
        #prediction=tf.argmax(y,1)
        #print(prediction.eval(feed_dict={x:[i[0] for i in test_data] }))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


AttributeError: 'int' object has no attribute 'reshape'

In [7]:
import tensorflow as tf
import numpy as np

IMG_SIZE_PX=50
SLICE_COUNT=20

n_classes=2

x=tf.placeholder('float')
y=tf.placeholder('float')

keep_rate=0.8
keep_prob=tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

def convolutional_neural_network(x):
    
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #'W_conv3':tf.Variable(tf.random_normal([3,3,3,64,128])),
               #'W_conv4':tf.Variable(tf.random_normal([5,5,5,128,256])),
               #'W_conv5':tf.Variable(tf.random_normal([5,5,5,256,512])),
               #'W_conv6':tf.Variable(tf.random_normal([5,5,5,512,1024])),
              # 'W_conv7':tf.Variable(tf.random_normal([5,5,5,1024,2048])),
               
               'W_fc':tf.Variable(tf.random_normal([ 54080 ,1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               #'b_conv3':tf.Variable(tf.random_normal([128])),
               #'b_conv4':tf.Variable(tf.random_normal([256])),
               #'b_conv5':tf.Variable(tf.random_normal([512])),
               #'b_conv6':tf.Variable(tf.random_normal([1024])),
               #'b_conv7':tf.Variable(tf.random_normal([2048])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    #conv3 = maxpool3d(conv3)
    #conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    #conv4 = maxpool3d(conv4)
    #conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    #conv5 = maxpool3d(conv5)
    #conv6 = tf.nn.relu(conv3d(conv5, weights['W_conv6']) + biases['b_conv6'])
    #conv6 = maxpool3d(conv6)
    #conv7 = tf.nn.relu(conv3d(conv6, weights['W_conv7']) + biases['b_conv7'])
    #conv7 = maxpool3d(conv7)
    
    

    fc = tf.reshape(conv2,[-1,  54080 ])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']
    

    return output
much_data = np.load('D:/muchdata-50-50-20.npy')
test_data=np.load('D:/testdata-50-50-20.npy')
train_data = much_data[-990:]
validation_data = much_data[-198:]
def train_neural_network(x):
    with tf.Session() as sess:
        prediction = convolutional_neural_network(x)
        cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
        optimizer = tf.train.AdamOptimizer().minimize(cost)
        saver = tf.train.Saver()
    
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        hm_epochs = 3
        
        for epoch in range(hm_epochs):
            for data in train_data:
                total_runs += 1
                epoch_loss = 0
                X = data[0]
                Y = data[1]
                _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                epoch_loss += c
                successful_runs += 1
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs) 
        saver.save(sess,'D:/3dconvnetmodel.ckpt')
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 3 loss: 47426440.0
Accuracy: 0.691919
Epoch 2 completed out of 3 loss: 0.0
Accuracy: 0.732323
Epoch 3 completed out of 3 loss: 0.0
Accuracy: 0.80303
Done. Finishing accuracy:
Accuracy: 0.772727
fitment percent: 1.0


In [31]:
with tf.Session as sess:
    saver=tf.train.Saver()
    saver.restore(sess,'D:/3dconvmodel.ckpt')
    sess.run(tf.initialize_all_variables())

AttributeError: __exit__